<a href="https://colab.research.google.com/github/pratik-poudel/jane-street/blob/main/5%20Feature%20Feature%20Feature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download gdonchyts/jane-street-market-prediction-parquet -f train.parquet
import zipfile
zip_ref = zipfile.ZipFile('/content/train.parquet.zip', 'r')
zip_ref.extractall('files')
zip_ref.close()

 99% 1.06G/1.08G [00:09<00:00, 76.8MB/s]
100% 1.08G/1.08G [00:09<00:00, 116MB/s] 


In [53]:
import numpy as np
import pandas as pd
from collections import defaultdict
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import gc
# pd.options.display.float_format| = "{:.2f}".format
pd.options.display.max_columns = 500

In [54]:
train = pd.read_parquet('/content/files/train.parquet')
train = train.query('date > 85').reset_index(drop = True) 
train = train[train['weight'] != 0]
train.shape

(1571415, 138)

In [55]:
train.fillna(train.mean(), inplace=True)

# Two Features

In [56]:
feat = [f for f in train.columns if 'feature' in f]

In [57]:
len(feat)

130

In [58]:
corr = train[feat].corrwith(train['resp']).abs().sort_values(ascending=False)

In [74]:
two_comb = []
from itertools import combinations
for i in combinations(corr.index.to_list(), 2):
    two_comb.append(i)

In [75]:
len(two_comb)

8385

In [76]:
two_comb[0]

('feature_39', 'feature_31')

In [77]:
from itertools import permutations,combinations_with_replacement, combinations
operators = ["+","-","*","/"]
for i in combinations_with_replacement(operators, 1):
    print(i)

('+',)
('-',)
('*',)
('/',)


In [78]:
two_comb[:5]

[('feature_39', 'feature_31'),
 ('feature_39', 'feature_27'),
 ('feature_39', 'feature_33'),
 ('feature_39', 'feature_35'),
 ('feature_39', 'feature_29')]

In [79]:
%%time
from google.colab import output

th = 0.02
feature_name =[]
score = []
count = 0
for i in two_comb:
    a = i[0]
    b = i[1]
    # c = i[2]
    
    train['computed'] = train[a] + train[b]
    cor = np.abs(train['computed'].corr(train['resp']))
    if cor > th:
        f_name = '+'.join([a, b])
        feature_name.append(f_name)
        score.append(cor)
    
    train['computed'] = train[a] - train[b]
    cor = np.abs(train['computed'].corr(train['resp']))
    if cor > th:
        f_name = '-'.join([a, b])
        feature_name.append(f_name)
        score.append(cor)
    
    train['computed'] = train[a] * train[b]
    cor = np.abs(train['computed'].corr(train['resp']))
    if cor > th:
        f_name = '*'.join([a, b])
        feature_name.append(f_name)
        score.append(cor)
    
    
    train['computed'] = train[a] / train[b]
    cor = np.abs(train['computed'].corr(train['resp']))
    if cor > th:
        f_name = '/'.join([a, b])
        feature_name.append(f_name)
        score.append(cor)

    train['computed'] = train[b] / train[a]
    cor = np.abs(train['computed'].corr(train['resp']))
    if cor > th:
        f_name = '/'.join([b, a])
        feature_name.append(f_name)
        score.append(cor)
   
    count += 1
    print(count, "done out of ", len(two_comb) )
    if count % 10 ==0:
        output.clear()

8381 done out of  8385
8382 done out of  8385
8383 done out of  8385
8384 done out of  8385
8385 done out of  8385
CPU times: user 27min 38s, sys: 58.3 s, total: 28min 36s
Wall time: 26min 34s


In [80]:
two_features = pd.DataFrame(zip(feature_name, score), columns=['features', 'score']).sort_values(by='score',ascending=False)

In [81]:
two_features

,features,score
104,feature_39*feature_96,0.058274
192,feature_39*feature_102,0.058261
233,feature_39*feature_116,0.057796
116,feature_39*feature_108,0.057319
225,feature_39*feature_114,0.057139
...,...,...
2113,feature_17*feature_60,0.020018
1236,feature_35-feature_57,0.020015
2221,feature_28+feature_111,0.020015
2382,feature_23-feature_61,0.020010


In [82]:
two_features.to_csv('two_features_normal.csv', index=False)

In [ ]:
# pd.DataFrame(zip(feature_name, score), columns=['features', 'score'])
# .sort_values(by='score',ascending=False)
# .to_csv('three_features.csv', index=False)

In [88]:
import re
feat_set = set()
for i in two_features[two_features['score'] > 0.055]['features'].to_list():
    f = re.split('[- + * /]',i)
    for j in f:
        feat_set.add(j)

In [90]:
df = train[feat_set]

In [95]:
to_split = two_features[two_features['score'] > 0.055]['features'].to_list()

feature_split = [] 
symbol_split = []
all_name = []
for i in to_split:
    all_name.append(i)
    features_list = re.split('[- + * /]',i)
    maths = re.findall(r"([\+\-\*\/]+)", i)
    feature_split.append(features_list)
    symbol_split.append(maths + [0])

In [97]:
feature_split

[['feature_39', 'feature_96'],
 ['feature_39', 'feature_102'],
 ['feature_39', 'feature_116'],
 ['feature_39', 'feature_108'],
 ['feature_39', 'feature_114'],
 ['feature_96', 'feature_0'],
 ['feature_0', 'feature_96'],
 ['feature_37', 'feature_84'],
 ['feature_39', 'feature_43'],
 ['feature_108', 'feature_0'],
 ['feature_0', 'feature_108']]

In [98]:
symbol_split

[['*', 0],
 ['*', 0],
 ['*', 0],
 ['*', 0],
 ['*', 0],
 ['/', 0],
 ['*', 0],
 ['*', 0],
 ['*', 0],
 ['/', 0],
 ['*', 0]]

In [103]:
count = 0
for i,j in zip(feature_split, symbol_split):
    feature_name = all_name[count]

    print("df['"+feature_name+"']", "= " "train["+"'"+i[0]+"'"+"]", str(j[0]), "train["+"'"+i[1]+"'"+"]")
    count += 1

df['feature_39*feature_96'] = train['feature_39'] * train['feature_96']
df['feature_39*feature_102'] = train['feature_39'] * train['feature_102']
df['feature_39*feature_116'] = train['feature_39'] * train['feature_116']
df['feature_39*feature_108'] = train['feature_39'] * train['feature_108']
df['feature_39*feature_114'] = train['feature_39'] * train['feature_114']
df['feature_96/feature_0'] = train['feature_96'] / train['feature_0']
df['feature_0*feature_96'] = train['feature_0'] * train['feature_96']
df['feature_37*feature_84'] = train['feature_37'] * train['feature_84']
df['feature_39*feature_43'] = train['feature_39'] * train['feature_43']
df['feature_108/feature_0'] = train['feature_108'] / train['feature_0']
df['feature_0*feature_108'] = train['feature_0'] * train['feature_108']


In [102]:
df = pd.DataFrame()

In [104]:
df['feature_39*feature_96'] = train['feature_39'] * train['feature_96']
df['feature_39*feature_102'] = train['feature_39'] * train['feature_102']
df['feature_39*feature_116'] = train['feature_39'] * train['feature_116']
df['feature_39*feature_108'] = train['feature_39'] * train['feature_108']
df['feature_39*feature_114'] = train['feature_39'] * train['feature_114']
df['feature_96/feature_0'] = train['feature_96'] / train['feature_0']
df['feature_0*feature_96'] = train['feature_0'] * train['feature_96']
df['feature_37*feature_84'] = train['feature_37'] * train['feature_84']
df['feature_39*feature_43'] = train['feature_39'] * train['feature_43']
df['feature_108/feature_0'] = train['feature_108'] / train['feature_0']
df['feature_0*feature_108'] = train['feature_0'] * train['feature_108']

In [105]:
df

,feature_39*feature_96,feature_39*feature_102,feature_39*feature_116,feature_39*feature_108,feature_39*feature_114,feature_96/feature_0,feature_0*feature_96,feature_37*feature_84,feature_39*feature_43,feature_108/feature_0,feature_0*feature_108
0,-0.087763,-0.035141,-0.041600,-0.087185,-0.040825,0.433895,0.433895,0.147266,0.272913,0.431037,0.431037
2,0.023988,0.009605,0.011371,0.023830,0.011159,-0.433895,-0.433895,-0.237182,-0.114664,-0.431037,-0.431037
3,-0.014631,-0.005858,-0.006935,-0.014535,-0.006806,0.433895,0.433895,0.254205,-0.060581,0.431037,0.431037
6,0.052316,0.020948,0.024798,0.051971,0.024336,-0.433895,-0.433895,-0.136723,-0.348820,-0.431037,-0.431037
7,0.044007,0.017621,0.020859,0.043717,0.020471,-0.433895,-0.433895,-0.317111,-0.093171,-0.431037,-0.431037
...,...,...,...,...,...,...,...,...,...,...,...
1862550,0.245172,5.690322,6.526240,0.532712,5.305742,-0.102147,-0.102147,-0.093927,-1.846862,-0.221946,-0.221946
1862552,-10.014939,-4.934894,-5.148413,-12.336777,-3.474740,1.668274,1.668274,1.460981,-25.799120,2.055042,2.055042
1862584,0.882326,-0.601499,0.266785,0.782531,-0.835405,-0.756951,-0.756951,-0.050074,-0.475259,-0.671336,-0.671336
1862587,1.400771,-2.053138,-1.860062,1.860842,-1.867003,-0.883252,-0.883252,-0.202110,-0.710345,-1.173349,-1.173349


In [108]:
df.corrwith(train['resp']).abs().sort_values(ascending=False)

feature_39*feature_96     0.058274
feature_39*feature_102    0.058261
feature_39*feature_116    0.057796
feature_39*feature_108    0.057319
feature_39*feature_114    0.057139
feature_0*feature_96      0.056833
feature_96/feature_0      0.056833
feature_37*feature_84     0.056099
feature_39*feature_43     0.055729
feature_0*feature_108     0.055203
feature_108/feature_0     0.055203
dtype: float64

# Repeat

In [115]:
two_comb = []
from itertools import combinations
for i in combinations(df.columns.to_list(), 2):
    two_comb.append(i)

In [ ]:
two_comb

In [120]:
%%time
from google.colab import output

th = 0.06
feature_name =[]
score = []
count = 0
for i in two_comb:
    a = i[0]
    b = i[1]
    # c = i[2]
    
    df['computed'] = df[a] + df[b]
    cor = np.abs(df['computed'].corr(train['resp']))
    if cor > th:
        f_name = '+'.join([a, b])
        feature_name.append(f_name)
        score.append(cor)
    
    df['computed'] = df[a] - df[b]
    cor = np.abs(df['computed'].corr(train['resp']))
    if cor > th:
        f_name = '-'.join([a, b])
        feature_name.append(f_name)
        score.append(cor)
    
    df['computed'] = df[a] * df[b]
    cor = np.abs(df['computed'].corr(train['resp']))
    if cor > th:
        f_name = '*'.join([a, b])
        feature_name.append(f_name)
        score.append(cor)
    
    
    df['computed'] = df[a] / df[b]
    cor = np.abs(df['computed'].corr(train['resp']))
    if cor > th:
        f_name = '/'.join([a, b])
        feature_name.append(f_name)
        score.append(cor)

    df['computed'] = df[b] / df[a]
    cor = np.abs(df['computed'].corr(train['resp']))
    if cor > th:
        f_name = '/'.join([b, a])
        feature_name.append(f_name)
        score.append(cor)
   
    count += 1
    print(count, "done out of ", len(two_comb) )
    if count % 10 ==0:
        output.clear()

51 done out of  55
52 done out of  55
53 done out of  55
54 done out of  55
55 done out of  55
CPU times: user 9.35 s, sys: 274 ms, total: 9.62 s
Wall time: 8.87 s


In [122]:
combined_features = pd.DataFrame(zip(feature_name, score), columns=['features', 'score']).sort_values(by='score',ascending=False)

In [123]:
to_split = combined_features[combined_features['score'] > 0.06]['features'].to_list()

feature_split = [] 
symbol_split = []
all_name = []
for i in to_split:
    all_name.append(i)
    features_list = re.split('[- + * /]',i)
    maths = re.findall(r"([\+\-\*\/]+)", i)
    feature_split.append(features_list)
    symbol_split.append(maths + [0])

In [130]:
feature_split

[['feature_37', 'feature_84', 'feature_39', 'feature_43'],
 ['feature_39', 'feature_116', 'feature_37', 'feature_84'],
 ['feature_0', 'feature_96', 'feature_39', 'feature_43'],
 ['feature_96', 'feature_0', 'feature_39', 'feature_43'],
 ['feature_39', 'feature_96', 'feature_39', 'feature_43'],
 ['feature_39', 'feature_102', 'feature_39', 'feature_43'],
 ['feature_39', 'feature_43', 'feature_0', 'feature_108'],
 ['feature_39', 'feature_43', 'feature_108', 'feature_0'],
 ['feature_39', 'feature_102', 'feature_37', 'feature_84'],
 ['feature_39', 'feature_116', 'feature_39', 'feature_43'],
 ['feature_39', 'feature_114', 'feature_37', 'feature_84'],
 ['feature_39', 'feature_114', 'feature_39', 'feature_43'],
 ['feature_39', 'feature_108', 'feature_39', 'feature_43'],
 ['feature_39', 'feature_96', 'feature_37', 'feature_84'],
 ['feature_39', 'feature_108', 'feature_37', 'feature_84'],
 ['feature_39', 'feature_116', 'feature_0', 'feature_96'],
 ['feature_39', 'feature_116', 'feature_96', 'feat

In [135]:
count = 0
for i,j in zip(feature_split, symbol_split):
    feature_name = all_name[count]

    print("df1['"+feature_name+"']", "= " "train["+"'"+i[0]+"'"+"]", str(j[0]), 
                                        "train["+"'"+i[1]+"'"+"]", str(j[1]),
                                        "train["+"'"+i[2]+"'"+"]", str(j[2]),
                                        "train["+"'"+i[3]+"'"+"]"
                                        )
    count += 1

df1['feature_37*feature_84-feature_39*feature_43'] = train['feature_37'] * train['feature_84'] - train['feature_39'] * train['feature_43']
df1['feature_39*feature_116-feature_37*feature_84'] = train['feature_39'] * train['feature_116'] - train['feature_37'] * train['feature_84']
df1['feature_0*feature_96-feature_39*feature_43'] = train['feature_0'] * train['feature_96'] - train['feature_39'] * train['feature_43']
df1['feature_96/feature_0-feature_39*feature_43'] = train['feature_96'] / train['feature_0'] - train['feature_39'] * train['feature_43']
df1['feature_39*feature_96+feature_39*feature_43'] = train['feature_39'] * train['feature_96'] + train['feature_39'] * train['feature_43']
df1['feature_39*feature_102+feature_39*feature_43'] = train['feature_39'] * train['feature_102'] + train['feature_39'] * train['feature_43']
df1['feature_39*feature_43-feature_0*feature_108'] = train['feature_39'] * train['feature_43'] - train['feature_0'] * train['feature_108']
df1['feature_39*feature_43-

In [136]:
df1 = pd.DataFrame()

In [137]:
df1['feature_37*feature_84-feature_39*feature_43'] = train['feature_37'] * train['feature_84'] - train['feature_39'] * train['feature_43']
df1['feature_39*feature_116-feature_37*feature_84'] = train['feature_39'] * train['feature_116'] - train['feature_37'] * train['feature_84']
df1['feature_0*feature_96-feature_39*feature_43'] = train['feature_0'] * train['feature_96'] - train['feature_39'] * train['feature_43']
df1['feature_96/feature_0-feature_39*feature_43'] = train['feature_96'] / train['feature_0'] - train['feature_39'] * train['feature_43']
df1['feature_39*feature_96+feature_39*feature_43'] = train['feature_39'] * train['feature_96'] + train['feature_39'] * train['feature_43']
df1['feature_39*feature_102+feature_39*feature_43'] = train['feature_39'] * train['feature_102'] + train['feature_39'] * train['feature_43']
df1['feature_39*feature_43-feature_0*feature_108'] = train['feature_39'] * train['feature_43'] - train['feature_0'] * train['feature_108']
df1['feature_39*feature_43-feature_108/feature_0'] = train['feature_39'] * train['feature_43'] - train['feature_108'] / train['feature_0']
df1['feature_39*feature_102-feature_37*feature_84'] = train['feature_39'] * train['feature_102'] - train['feature_37'] * train['feature_84']
df1['feature_39*feature_116+feature_39*feature_43'] = train['feature_39'] * train['feature_116'] + train['feature_39'] * train['feature_43']
df1['feature_39*feature_114-feature_37*feature_84'] = train['feature_39'] * train['feature_114'] - train['feature_37'] * train['feature_84']
df1['feature_39*feature_114+feature_39*feature_43'] = train['feature_39'] * train['feature_114'] + train['feature_39'] * train['feature_43']
df1['feature_39*feature_108+feature_39*feature_43'] = train['feature_39'] * train['feature_108'] + train['feature_39'] * train['feature_43']
df1['feature_39*feature_96-feature_37*feature_84'] = train['feature_39'] * train['feature_96'] - train['feature_37'] * train['feature_84']
df1['feature_39*feature_108-feature_37*feature_84'] = train['feature_39'] * train['feature_108'] - train['feature_37'] * train['feature_84']
df1['feature_39*feature_116-feature_0*feature_96'] = train['feature_39'] * train['feature_116'] - train['feature_0'] * train['feature_96']
df1['feature_39*feature_116-feature_96/feature_0'] = train['feature_39'] * train['feature_116'] - train['feature_96'] / train['feature_0']
df1['feature_39*feature_116-feature_108/feature_0'] = train['feature_39'] * train['feature_116'] - train['feature_108'] / train['feature_0']
df1['feature_39*feature_116-feature_0*feature_108'] = train['feature_39'] * train['feature_116'] - train['feature_0'] * train['feature_108']
df1['feature_39*feature_96+feature_39*feature_116'] = train['feature_39'] * train['feature_96'] + train['feature_39'] * train['feature_116']
df1['feature_39*feature_102-feature_0*feature_96'] = train['feature_39'] * train['feature_102'] - train['feature_0'] * train['feature_96']
df1['feature_39*feature_102-feature_96/feature_0'] = train['feature_39'] * train['feature_102'] - train['feature_96'] / train['feature_0']
df1['feature_39*feature_102-feature_0*feature_108'] = train['feature_39'] * train['feature_102'] - train['feature_0'] * train['feature_108']
df1['feature_39*feature_102-feature_108/feature_0'] = train['feature_39'] * train['feature_102'] - train['feature_108'] / train['feature_0']
df1['feature_39*feature_114-feature_0*feature_96'] = train['feature_39'] * train['feature_114'] - train['feature_0'] * train['feature_96']
df1['feature_39*feature_114-feature_96/feature_0'] = train['feature_39'] * train['feature_114'] - train['feature_96'] / train['feature_0']
df1['feature_39*feature_116+feature_39*feature_108'] = train['feature_39'] * train['feature_116'] + train['feature_39'] * train['feature_108']
df1['feature_39*feature_114-feature_108/feature_0'] = train['feature_39'] * train['feature_114'] - train['feature_108'] / train['feature_0']
df1['feature_39*feature_114-feature_0*feature_108'] = train['feature_39'] * train['feature_114'] - train['feature_0'] * train['feature_108']
df1['feature_39*feature_96+feature_39*feature_102'] = train['feature_39'] * train['feature_96'] + train['feature_39'] * train['feature_102']
df1['feature_39*feature_96+feature_39*feature_114'] = train['feature_39'] * train['feature_96'] + train['feature_39'] * train['feature_114']
df1['feature_39*feature_102+feature_39*feature_108'] = train['feature_39'] * train['feature_102'] + train['feature_39'] * train['feature_108']
df1['feature_39*feature_96-feature_0*feature_96'] = train['feature_39'] * train['feature_96'] - train['feature_0'] * train['feature_96']
df1['feature_39*feature_96-feature_96/feature_0'] = train['feature_39'] * train['feature_96'] - train['feature_96'] / train['feature_0']
df1['feature_39*feature_102+feature_39*feature_116'] = train['feature_39'] * train['feature_102'] + train['feature_39'] * train['feature_116']
df1['feature_39*feature_96-feature_0*feature_108'] = train['feature_39'] * train['feature_96'] - train['feature_0'] * train['feature_108']
df1['feature_39*feature_96-feature_108/feature_0'] = train['feature_39'] * train['feature_96'] - train['feature_108'] / train['feature_0']

In [138]:
df1.corrwith(train['resp']).abs().sort_values(ascending=False)

feature_37*feature_84-feature_39*feature_43      0.071588
feature_39*feature_116-feature_37*feature_84     0.069503
feature_0*feature_96-feature_39*feature_43       0.068152
feature_96/feature_0-feature_39*feature_43       0.068152
feature_39*feature_96+feature_39*feature_43      0.067886
feature_39*feature_102+feature_39*feature_43     0.067782
feature_39*feature_43-feature_108/feature_0      0.067740
feature_39*feature_43-feature_0*feature_108      0.067740
feature_39*feature_102-feature_37*feature_84     0.067734
feature_39*feature_116+feature_39*feature_43     0.067553
feature_39*feature_114-feature_37*feature_84     0.067415
feature_39*feature_114+feature_39*feature_43     0.066493
feature_39*feature_108+feature_39*feature_43     0.066195
feature_39*feature_96-feature_37*feature_84      0.064953
feature_39*feature_108-feature_37*feature_84     0.064868
feature_39*feature_116-feature_0*feature_96      0.064287
feature_39*feature_116-feature_96/feature_0      0.064287
feature_39*fea

In [ ]:
combined_features

In [ ]:
t

# REPEAT 1

In [140]:
two_comb = []
from itertools import combinations
for i in combinations(df1.columns.to_list(), 2):
    two_comb.append(i)

In [145]:
len(two_comb)

666

In [146]:

def two_combinations(df):
    from google.colab import output
    th = 0.06
    feature_name =[]
    score = []
    count = 0
    for i in two_comb:
        a = i[0]
        b = i[1]
        # c = i[2]
        
        df['computed'] = df[a] + df[b]
        cor = np.abs(df['computed'].corr(train['resp']))
        if cor > th:
            f_name = '+'.join([a, b])
            feature_name.append(f_name)
            score.append(cor)
        
        df['computed'] = df[a] - df[b]
        cor = np.abs(df['computed'].corr(train['resp']))
        if cor > th:
            f_name = '-'.join([a, b])
            feature_name.append(f_name)
            score.append(cor)
        
        df['computed'] = df[a] * df[b]
        cor = np.abs(df['computed'].corr(train['resp']))
        if cor > th:
            f_name = '*'.join([a, b])
            feature_name.append(f_name)
            score.append(cor)
        
        
        df['computed'] = df[a] / df[b]
        cor = np.abs(df['computed'].corr(train['resp']))
        if cor > th:
            f_name = '/'.join([a, b])
            feature_name.append(f_name)
            score.append(cor)

        df['computed'] = df[b] / df[a]
        cor = np.abs(df['computed'].corr(train['resp']))
        if cor > th:
            f_name = '/'.join([b, a])
            feature_name.append(f_name)
            score.append(cor)
    
        count += 1
        print(count, "done out of ", len(two_comb) )
        if count % 10 ==0:
            output.clear()
    result = pd.DataFrame(zip(feature_name, score), columns=['features', 'score']).sort_values(by='score',ascending=False)
    return result

In [147]:
%%time 
result2 = two_combinations(df1)

661 done out of  666
662 done out of  666
663 done out of  666
664 done out of  666
665 done out of  666
666 done out of  666
CPU times: user 2min 14s, sys: 6.29 s, total: 2min 20s
Wall time: 2min 11s


In [ ]:
for i in result2['features']:
    print(i)

In [153]:
result2[result2['score'] > 0.076]['features'].to_list()

['feature_39*feature_116-feature_37*feature_84-feature_0*feature_96-feature_39*feature_43',
 'feature_39*feature_116-feature_37*feature_84-feature_96/feature_0-feature_39*feature_43',
 'feature_37*feature_84-feature_39*feature_43-feature_39*feature_116-feature_96/feature_0',
 'feature_37*feature_84-feature_39*feature_43-feature_39*feature_116-feature_0*feature_96',
 'feature_37*feature_84-feature_39*feature_43-feature_39*feature_116-feature_0*feature_108',
 'feature_37*feature_84-feature_39*feature_43-feature_39*feature_116-feature_108/feature_0',
 'feature_39*feature_116-feature_37*feature_84+feature_39*feature_43-feature_108/feature_0',
 'feature_39*feature_116-feature_37*feature_84+feature_39*feature_43-feature_0*feature_108',
 'feature_37*feature_84-feature_39*feature_43-feature_39*feature_116+feature_39*feature_43']